In [2]:
import pandas as pd
import numpy as np


In [131]:
# load data
url = "https://raw.githubusercontent.com/nikhil-bhargava/ids-705-final-project/main/data/raw/worldcities.csv"
city = pd.read_csv(url)
df = pd.read_csv('/Users/롸/ids-705-final-project/data/raw/train.csv')

In [132]:
# select relevant cities
city_df = city[city['id'].astype('str').isin(['1840034016', '1840020491', '1840000494', 
                                '1840006060', '1840000455', '1840021543'])]
city_df = city_df.rename(columns={'id':'city_id', 'city':'city_name'})
city_df['city_id'] = city_df['city_id'].astype('str')

In [133]:
bnb_city_id = {'NYC':'1840034016', 'SF':'1840021543', 'DC':'1840006060', 'LA':'1840020491', 'Chicago':'1840000494', 
               'Boston':'1840000455'}

df['city_id'] = df['city'].map(bnb_city_id)

In [134]:
#city_df= city_df[['city_id', 'lat', 'lng']]

In [135]:
new_df = pd.merge(df, city_df, how='left', on='city_id')

In [136]:
new_df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,city_name,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,...,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18713220.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,...,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18713220.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18713220.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,...,San Francisco,San Francisco,37.7562,-122.4430,United States,US,USA,California,NaN,3592294.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,...,Washington,Washington,38.9047,-77.0163,United States,US,USA,District of Columbia,primary,5379184.0


In [154]:
new_df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,admin_name,capital,population,cc_dist,num_bnb_1km,desc_len,name_len,bed_bath_ratio,years_of_hosting,years_since_lastreview
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,...,New York,NaN,18713220.0,3.498384,272,211,30,1.0,9.0,4.7
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,...,New York,NaN,18713220.0,5.991584,1400,1000,40,3.0,3.8,3.5
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,New York,NaN,18713220.0,7.925215,891,1000,16,1.0,4.4,3.5
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,...,California,NaN,3592294.0,1.256540,666,468,34,2.0,6.0,NaN
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,...,District of Columbia,primary,5379184.0,1.748733,727,699,26,0.0,6.1,4.2


In [155]:
new_df.drop('iso3', axis=1, inplace=True)

In [157]:
new_df.drop('iso2', axis=1, inplace=True)

In [158]:
new_df.drop('city_ascii', axis=1, inplace=True)

## Feature Engineering

#### Distance of Airbnb to city center


In [129]:
from geopy import distance

def dist_city_center(prop_lat, prop_lon, city_lat, city_lon):
    prop_coords = (prop_lat, prop_lon)
    city_coords = (city_lat, city_lon)
    dist = distance.great_circle(prop_coords, city_coords).miles
    return dist


In [138]:
new_df['cc_dist'] = new_df.apply(lambda x: dist_city_center(x.latitude, x.longitude, x.lat, x.lng), axis=1)


In [139]:

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
#     m = km * 0.621371
    
    return km


In [140]:
def num_close_airbnbs(y, df, dist_thresh):
    df['cur_lat'] = y.latitude
    df['cur_lon'] = y.longitude
    df['rel_dist'] = haversine_np(df['cur_lon'], df['cur_lat'], df['longitude'], df['latitude'])
    num_close = df[df['rel_dist']<=dist_thresh]
    return len(num_close)-1

In [182]:
# takes ~10 mins to run
df = new_df.copy()
new_df['num_bnb_1km'] = new_df.apply(lambda y: num_close_airbnbs(y, df, 1), axis=1)


#### Description Length

In [143]:
new_df['desc_len'] = new_df['description'].str.len()
new_df['name_len'] = new_df['name'].str.len()


#### Ratio of Bedrooms to bathrooms

In [144]:
new_df['bed_bath_ratio'] = round(new_df.bedrooms/new_df.bathrooms,2)


#### How long has the host been hosting

In [263]:
new_df['today'] = '2021-04-01'


In [264]:
new_df['host_since'] = pd.to_datetime(new_df['host_since'])
new_df['today'] = pd.to_datetime(new_df['today'])
new_df['years_of_hosting'] = round(((new_df['today'] - new_df['host_since']).dt.days)/365,1)


#### Years since last review (benchmark date = 2021-04-01)

In [265]:
new_df['last_review'] = pd.to_datetime(new_df['last_review'])
new_df['years_since_lastreview'] = round(((new_df['today'] - new_df['last_review']).dt.days)/365,1)


In [266]:
# Dropping the column I made to get the dates
new_df.drop('today', axis=1, inplace=True)


In [267]:
# null values generated because there is no 'last_review'
new_df.years_since_lastreview.isnull().sum()


0

#### Change % to float for column 'host_response_rate'

In [150]:
new_df['host_response_rate'] = new_df['host_response_rate'].str.rstrip('%').astype('float')


In [159]:
new_df.shape

(74111, 44)

#### Column 'host_has_profile_pic', 'host_since', 'host_identity_verified' all has 203 missing values

In [160]:
# Let's change all nan values to 'f' (no profile pic, identity not verified)
new_df.loc[new_df['host_has_profile_pic'].isnull(), 'host_has_profile_pic'] = 'f'
new_df.loc[new_df['host_identity_verified'].isnull(), 'host_identity_verified'] = 'f'


#### Encode 'thumbnail_url' column to 1 or 0 (if you have thumbnail then 1 else 0)

In [161]:
# we won't using this as dummy variable
new_df.thumbnail_url.nunique()


65883

In [162]:
# If the value is not null, then we assign 1, else 0
new_df.loc[new_df['thumbnail_url'].notnull(), 'thumbnail_url'] = 1


In [163]:
new_df.loc[new_df['thumbnail_url'].isnull(), 'thumbnail_url'] = 0


In [164]:
# Now we have either 1 or 0
new_df.thumbnail_url.unique()


array([1, 0], dtype=object)

#### Drop duplicate rows

In [165]:
duplicate_rows_new_df = new_df[new_df.duplicated()]
print('number of duplicate rows:', duplicate_rows_new_df.shape)

#drop the duplicate rows
new_df = new_df.drop_duplicates()



number of duplicate rows: (0, 44)


#### Missing values for bedrooms and bathrooms

In [166]:
# Since most of the airbnbs have 1 bedroom, i will impute the missing values with 1
new_df.bedrooms.value_counts()


1.0     49784
2.0     11351
0.0      6715
3.0      4309
4.0      1330
5.0       355
6.0       106
7.0        38
8.0        14
10.0       10
9.0         8
Name: bedrooms, dtype: int64

In [167]:
# Since most of the airbnbs have 1 bathroom, i will impute the missing values with 1
new_df.bathrooms.value_counts()


1.0    58099
2.0     7936
1.5     3801
2.5     1567
3.0     1066
3.5      429
4.0      286
0.5      209
0.0      198
4.5      116
5.0       72
8.0       41
5.5       39
6.0       24
6.5       12
7.0       10
7.5        6
Name: bathrooms, dtype: int64

In [168]:
new_df.beds.value_counts()


1.0     45144
2.0     16704
3.0      6442
4.0      3065
5.0      1287
6.0       661
7.0       216
8.0       186
10.0       88
9.0        72
16.0       38
12.0       29
11.0       23
13.0       10
15.0        6
14.0        4
0.0         4
18.0        1
Name: beds, dtype: int64

In [169]:
new_df['bathrooms'].fillna(new_df['bathrooms'].mode()[0], inplace=True)
new_df['bedrooms'].fillna(new_df['bedrooms'].mode()[0], inplace=True)
new_df['beds'].fillna(new_df['beds'].mode()[0], inplace=True)


#### Number of Beds to bedroom ratio

In [170]:
new_df['bed_bath_ratio'] = round(new_df.bedrooms/new_df.bathrooms,2)


In [171]:
# There are null values in 'bed_bath_ratio' because there are houses with no bathrooms (campers, private rooms..)
# Replace null values in 'bed_bath_ratio' with 0
new_df.loc[new_df['bed_bath_ratio'].isnull(), 'bed_bath_ratio'] = 0.00


#### Drop 'neighborhood', 'zipcode', 'lat', 'lng', 'description' and 'name'

In [172]:
new_df.drop('zipcode', axis=1, inplace=True)
new_df.drop('neighbourhood', axis=1, inplace=True)

In [173]:
new_df.drop('lat', axis=1, inplace=True)
new_df.drop('lng', axis=1, inplace=True)

In [174]:
new_df.drop('name', axis=1, inplace=True)

In [175]:
new_df.drop('description', axis=1, inplace=True)

#### Drop bed_type, cancellation_policy, cleaning_fee (not in test)


In [185]:
new_df.drop('bed_type', axis=1, inplace=True)

In [186]:
new_df.drop('cancellation_policy', axis=1, inplace=True)

In [187]:
new_df.drop('cleaning_fee', axis=1, inplace=True)

In [192]:
new_df.drop('capital', axis=1, inplace=True)

In [193]:
new_df.drop('admin_name', axis=1, inplace=True)

In [196]:
new_df.drop('population', axis=1, inplace=True)

In [212]:
new_df.drop('city_id', axis=1, inplace=True)

In [213]:
new_df.drop('latitude', axis=1, inplace=True)

In [214]:
new_df.drop('longitude', axis=1, inplace=True)

In [221]:
new_df.drop('country', axis=1, inplace=True)

In [226]:
new_df.drop('city', axis=1, inplace=True)

In [227]:
new_df.drop('city_name', axis=1, inplace=True)

#### Amenities

In [195]:
def handle_entry(s):
    # remove curly brace
    s = s.strip('{').strip('}')
    s = s.lower()
    # split
    s_list = s.split(",")
    # remove extra quote
    for i, curr_s in enumerate(s_list):
        s_list[i] = s_list[i].strip('"')
    return s_list

In [197]:
new_df['amenities'] = new_df['amenities'].apply(lambda x: handle_entry(x))

In [198]:
amenity_dict = {'amenity_tv': ['cable tv','tv'], 
 'amenity_baby': ['baby bath','baby monitor', 'babysitter recommendations', 'changing table', 
                  "children’s books and toys", "children’s dinnerware", 'family/kid friendly',
                  'fireplace guards', 'crib', 'high chair', "pack ’n play/travel crib", 'table corner guards',
                  'outlet covers', 'stair gates'],
'amenity_bed': ['bed linens', "extra pillows and blankets",'firm mattress','firm matress','firm mattress'],
'amenity_accessibility': [' smooth pathway to front door','accessible-height bed','accessible-height toilet',
                          'disabled parking spot','fixed grab bars for shower & toilet','flat smooth pathway to front door',
                          'grab-rails for shower and toilet','ground floor access','handheld shower head',
                          'path to entrance lit at night','roll-in shower with chair','step-free access',
                          'well-lit path to entrance','wheelchair accessible','wide clearance to bed',
                          'wide clearance to shower & toilet','wide clearance to shower and toilet','wide doorway',
                          'wide entryway','wide hallway clearance'],
'amenity_aircondition': ['air conditioning'],
'amenity_bbqgrill': ['bbq grill'], 
'amenity_breakfast': ['breakfast'],
'amenity_buzzer_wireless_intercom':['buzzer/wireless intercom'],
'amenity_cleaning_bf_checkout': ['cleaning before checkout'],
'amenity_pets': ['cat(s)', 'dog(s)', 'other pet(s)', 'pets allowed', 'pets live on this property'],
'amenity_washer_dryer':['dryer','washer','washer / dryer'],
'amenity_ev_charger':['ev charger'],
'amenity_elevator':['elevator','elevator in building'],
'amenity_free_park':['free parking on premises','free parking on street'],
'amenity_game':['game console'],
'amenity_indoor_fireplace': ['indoor fireplace'],
'amenity_gym':['gym'],
'amenity_clothes':['iron', 'hangers'],
'amenity_essential':['essentials'],
'amenity_hot_water': ['hot water'],
'amenity_garden_backyard':['garden or backyard'],
'amenity_bathroom':['bath towel', 'hand or paper towel', 'toilet paper', 'hand soap', 'body soap', 
                    'bathtub', 'shampoo', 'bathtub with shower chair', 'hot tub', 'shampoo', 'hair dryer'],
'amenity_host_greet': ['host greets you'], 
'amenity_heating': ['heating'],
'amenity_kitchen': ['refrigerator', 'dishes and silverware', 'microwave', 'hot water kettle', 
                    'dishwasher', 'coffee maker', 'stove', 'oven', 'cooking basics'],
'amenity_lake': ['lake access'],
'amenity_work': ['laptop friendly workspace'],
'amenity_bedroom_lock': ['lock on bedroom door'],
'amenity_lockbox': ['lockbox', 'keypad'],
'amenity_longterm_stays': ['long term stays allowed'],
'amenity_luggage_dropoff': ['luggage dropoff allowed'],
'amenity_other': ['other'],
'amenity_paid_parking': ['paid parking off premises'],
'amenity_patio_balcony': ['patio or balcony'],
'amenity_wifi': ['pocket wifi', 'wireless internet', 'internet', 'ethernet connection', 'internet'],
'amenity_pool': ['pool'],
'amenity_privacy': ['private bathroom', 'private entrance', 'private living room', 'room-darkening shades'],
'amenity_safety': ['safety card', 'smart lock', 'smartlock', 'smoke detector', 
                   'fire extinguisher', 'first aid kit', 'window guards'],
'amenity_self_checkin': ['self check-in'],
'amenity_single_level_home': ['single level home'],
'amenity_smoking_allowed': ['smoking allowed'],
'amenity_suitable_for_events': ['suitable for events'],
'amenity_waterfront': ['waterfront'],
'amenity_window_guards': ['window guards'],
'amenity_self_checkin':['self check-in'],
'amenity_beach_essentials':['beach essentials'],
'amenity_beach_front':['beachfront'],
'amenity_doorman':['doorman','doorman entry'],
'amenity_24hour_checkin':['24-hour check-in']}

In [199]:
def ohe_amenities(amenity, x):
    if amenity in x:
        return 1
    else:
        return 0

In [200]:
for agroup in amenity_dict.keys():
    amenity_cols = amenity_dict[agroup]
    for amenity in amenity_cols:
        new_df[amenity] = new_df['amenities'].apply(lambda x: ohe_amenities(amenity, x))
    new_df[agroup] = new_df[amenity_cols].sum(axis=1)
    new_df = new_df.drop(columns=amenity_cols)
    new_df[agroup] = np.where(new_df[agroup] >= 1, 1, 0)

In [232]:
new_df = new_df.fillna(method='bfill')

In [233]:
new_df.first_review.isnull().sum()

0

In [234]:
new_df.last_review.isnull().sum()

0

In [237]:
new_df.host_since.isnull().sum()

0

In [238]:
new_df.host_response_rate.isnull().sum()

0

#### One-hot-encoding

In [240]:
cate_vari = ['property_type','room_type',
             'host_has_profile_pic','host_identity_verified','instant_bookable']
for i in cate_vari:
    #train[i]=train[i].astype(str)
    new_df[i]=pd.Categorical(new_df[i])

In [242]:
#one hot encoding for categorical variables
from sklearn.preprocessing import OneHotEncoder

# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')

# passing column (label encoded values)
df_ohe = pd.DataFrame(enc.fit_transform(new_df[cate_vari]).toarray())

df_ohe.columns = enc.get_feature_names()

# merge with main df bridge_df on key values
merged_df = new_df.join(df_ohe)


merged_df.drop(columns=cate_vari, inplace=True)


In [248]:
merged_df.drop('id', axis=1, inplace=True)

In [250]:
merged_df.drop('amenities', axis=1, inplace=True)

In [268]:
merged_df.drop('first_review', axis=1, inplace=True)

In [269]:
merged_df.drop('last_review', axis=1, inplace=True)

In [270]:
merged_df.drop('host_since', axis=1, inplace=True)

In [271]:
merged_df.head()

,log_price,accommodates,bathrooms,host_response_rate,number_of_reviews,review_scores_rating,thumbnail_url,bedrooms,beds,cc_dist,...,x0_Yurt,x1_Entire home/apt,x1_Private room,x1_Shared room,x2_f,x2_t,x3_f,x3_t,x4_f,x4_t
0,5.010635,3,1.0,100.0,2,100.0,1,1.0,1.0,3.498384,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,5.129899,7,1.0,100.0,6,93.0,1,3.0,3.0,5.991584,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,4.976734,5,1.0,100.0,10,92.0,1,1.0,3.0,7.925215,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,6.620073,4,1.0,100.0,0,40.0,1,2.0,2.0,1.256540,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,4.744932,2,1.0,100.0,4,40.0,0,0.0,1.0,1.748733,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [272]:
from sklearn.model_selection import train_test_split

X = merged_df
X = merged_df.drop(columns=['log_price'])

y = merged_df.log_price

In [273]:
X.dtypes

accommodates              int64
bathrooms               float64
host_response_rate      float64
number_of_reviews         int64
review_scores_rating    float64
                         ...   
x2_t                    float64
x3_f                    float64
x3_t                    float64
x4_f                    float64
x4_t                    float64
Length: 107, dtype: object

In [278]:
X.isnull().any().sum()

0

In [279]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

ValueError: Input contains infinity or a value too large for dtype('float64').

#### PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import (KNeighborsClassifier, NeighborhoodComponentsAnalysis)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


#figure out how much dimension we need after pca
pca = PCA(n_components=X_train.shape[1])
pca_transform = pca.fit_transform(X_train,y_train)
var_values = pca.explained_variance_ratio_


cum_exp_variance = np.cumsum(var_values)
plt.plot(cum_exp_variance)
plt.xlabel('principal components of the data')
plt.ylabel('fraction of variance explained')
plt.title('PCA cumulative fraction of variance explained')

cum_exp_variance

#### Save it as csv 

In [189]:
df.to_csv('/Users/롸/ids-705-final-project/cleaned.csv')

## Model

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor


## 1. Linear Regression

In [ ]:

lr = LinearRegression()
lr.fit(X_train, y_train)


print('Train R^2', lr.score(X_train,y_train))
print('Val R^2', lr.score(X_val,y_val))

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

In [ ]:
y_pred_lr = lr.predict(X_val)
mean_squared_error(y_val, y_pred_lr)

## 2. Random Forest

In [ ]:
rf = RandomForestRegressor(max_depth=5)
rf.fit(X_train, y_train)

print('Train R^2', rf.score(X_train,y_train))
print('Val R^2', rf.score(X_val,y_val))

In [ ]:
mean_squared_error(y_train, rf.predict(X_train))

In [ ]:
y_pred_rf = rf.predict(X_val)
mean_squared_error(y_val, y_pred_rf)

In [ ]:

nn = MLPRegressor(learning_rate_init=0.05)
nn.fit(X_train, y_train)

print('Train R^2', nn.score(X_train,y_train))
print('Val R^2', nn.score(X_val,y_val))

In [ ]:
mean_squared_error(y_train, nn.predict(X_train))


In [ ]:
y_pred_nn = nn.predict(X_val)
mean_squared_error(y_val, y_pred_nn)

## 3. Neural Net

## 4. XGBoost

## 5. LightGBM